### Описание задачи
Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.  

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, **1,2,3,4;** или **1,2,3,4;5,6**.

Гарантируется, что среди id купленных товаров все различные.

**Важно:**

- Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
- Если товар не встречался в обучающей выборке, его популярность равна 0.
- Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
- Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

**Задание**

На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

Реализуйте два алгоритма рекомендаций:
- сортировка просмотренных id по популярности (частота появления в просмотренных),
- сортировка просмотренных id по покупаемости (частота появления в покупках).

Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
data = pd.read_csv("coursera_sessions_train.txt", delimiter=";", names=["viewed", "purchased"])
data.viewed = data.viewed.str.split(',')
data.purchased = data.purchased.str.split(',')

In [3]:
data_test = pd.read_csv("coursera_sessions_test.txt", delimiter=";", names=["viewed", "purchased"])
data_test.viewed = data_test.viewed.str.split(',')
data_test.purchased = data_test.purchased.str.split(',')
data_test.dropna(inplace=True)
data_test.reset_index(inplace=True, drop=True)

In [4]:
viewed_counter = Counter()

for session in data.viewed:
    for viewed_id in session:
        viewed_counter[viewed_id] += 1

viewed_counter.most_common(5)

[('73', 677), ('158', 641), ('204', 396), ('262', 387), ('162', 318)]

In [5]:
data.dropna(inplace=True)
data.reset_index(inplace=True, drop=True)

In [6]:
purchased_counter = Counter()

for session in data.purchased:
    for purchased_id in session:
        purchased_counter[purchased_id] += 1

[('158', 14), ('204', 12), ('3324', 11), ('73', 11), ('5569', 10)]

In [7]:
def avg_recall_precision(data, k, counter):
    precisions = []
    recalls = []
    for i, session in data.iterrows():
        session_recomendations = pd.unique(session.viewed).tolist()
        session_recomendations.sort(key=lambda x: -counter[x])
        session_recomendations = session_recomendations[:min(k, len(session_recomendations))]        
        
        used_recomendations = [x for x in session.purchased if x in session_recomendations]
        precisions.append(len(used_recomendations)/k)
        recalls.append(len(used_recomendations)/len(session.purchased))
    return  np.mean(recalls), np.mean(precisions)

In [8]:
def run_experiment(data, counter, file_name = None):
    k1 = avg_recall_precision(data, 1, counter)
    k5 = avg_recall_precision(data, 5, counter)
    results = [round(i,2) for sub in [k1, k5] for i in sub]
    if file_name is not None:
        result = " ".join([str(x) for x in result])
        with open(file_name, "w+") as file:
            file.write(result)
    return results

In [9]:
results = run_experiment(data, viewed_counter)
print('Test set. ')

[0.44, 0.51, 0.82, 0.21]

In [10]:
run_experiment(data, purchased_counter)

[0.69, 0.8, 0.93, 0.25]

In [11]:
run_experiment(data_test, viewed_counter)


[0.42, 0.48, 0.8, 0.2]

In [12]:
run_experiment(data_test, purchased_counter)


[0.46, 0.53, 0.82, 0.21]